In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[2],2
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[-2, -2, 0]"
ng_refs,-,"[2, 3, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.118900                    1  0.059808
0.000750        2  0.119818                    2  0.060831
0.001052        3  0.123301                    3  0.064058
0.001476        4  0.126628                    4  0.066964
0.002070        5  0.129548                    5  0.069855
0.002904        6  0.132105                    6  0.072658
0.004074        7  0.133894                    7  0.075356
0.005714        8  0.134868                    8  0.077843
0.008015        9  0.134708                    9  0.079996
0.011243       10  0.133098                   10  0.081699
0.015771       11  0.129701                   11  0.082721
0.022122       12  0.124028                   12  0.082703
0.031031       13  0.115601                   13  0.081179
0.043528       14  0.104960                   14  0.078548
0.061057       15  0.105091                   15  0.085203
0.085645       16  0.122690                   16  0.107263
0.120136       17  0.144798                   17  0.134372
0.168516       18  0.162870                   18  0.157483
0.236378       19  0.177022                   19  0.173828
0.331549       20  0.189644                   20  0.184525
0.465100       21  0.206032                   21  0.198296
0.652400       22  0.227466                   22  0.222723
0.915100       23  0.211525                   23  0.215994
1.283650       24  0.169448                   24  0.181174
1.800600       25  0.136518                   25  0.148720
2.525700       26  0.109344                   26  0.116659
3.542800       27  0.091800                   27  0.094673
4.969550       28  0.080593                   28  0.080924
6.970850       29  0.072098                   29  0.071708
9.778100       30  0.064670                   30  0.062673
13.715850      31  0.057762                   31  0.054276
19.239350      32  0.051205                   32  0.047922
26.987250      33  0.044974                   33  0.043185
37.855300      34  0.039068                   34  0.037585
53.100050      35  0.028492                   35  0.026182
73.887500      36  0.013776                   36  0.010896
97.662500      37  0.004116                   37  0.001446
121.437500     38  0.007674                   38  0.005541
145.212500     39  0.020978                   39  0.019427
168.987500     40  0.036753                   40  0.036228
192.762500     41  0.070259                   41  0.071826
216.537500     42  0.166599                   42  0.173063
240.312500     43  0.292787                   43  0.289025
264.087500     44  0.433296                   44  0.404623
287.862500     45  0.575545                   45  0.541741
311.637500     46  0.706863                   46  0.662589
335.412500     47  0.817635                   47  0.737379
359.187500     48  0.898222                   48  0.826422
382.962500     49  0.924448                   49  0.884820
406.737500     50  0.967756                   50  0.929765
430.512500     51  1.015138                   51  0.977660
454.287500     52  1.056514                   52  1.042155
478.062500     53  1.085549                   53  1.103134
501.837500     54  1.097525                   54  1.129018
525.612500     55  1.093802                   55  1.123971
549.387500     56  1.073029                   56  1.098541
573.162500     57  1.036783                   57  1.072343
596.937500     58  0.987585                   58  1.046464
620.712500     59  0.927590                   59  1.007610
644.487500     60  0.862490                   60  0.947289
668.262500     61  0.796197                   61  0.863719
692.037500     62  0.727999                   62  0.763375
715.812500     63  0.651818                   63  0.654569
739.587500     64  0.589967                   64  0.570638
763.362500     65  0.490411                   65  0.465554
78

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -61.195825   0.000000 -61.195825 -60.873407   0.000002   
0.000624    2     -61.195830   0.000014 -61.195816 -60.873412   0.000012   
0.000876    3     -61.195832   0.000020 -61.195812 -60.873414   0.000016   
0.001229    4     -61.195835   0.000028 -61.195807 -60.873418   0.000022   
0.001723    5     -61.195840   0.000040 -61.195800 -60.873422   0.000030   
0.002417    6     -61.195846   0.000056 -61.195789 -60.873428   0.000042   
0.003391    7     -61.195854   0.000080 -61.195774 -60.873436   0.000059   
0.004757    8     -61.195865   0.000113 -61.195752 -60.873448   0.000083   
0.006672    9     -61.195881   0.000159 -61.195722 -60.873464   0.000117   
0.009359    10    -61.195903   0.000224 -61.195679 -60.873487   0.000165   
0.013128    11    -61.195934   0.000315 -61.195619 -60.873519   0.000234   
0.018415    12    -61.195978   0.000439 -61.195538 -60.873564   0.000330   
0.025830    13    -61.196040   0.000610 -61.195429 -60.873628   0.000467   
0.036232    14    -61.196129   0.000841 -61.195287 -60.873718   0.000657   
0.050823    15    -61.196257   0.001151 -61.195105 -60.873848   0.000922   
0.071291    16    -61.196439   0.001588 -61.194851 -60.874029   0.001310   
0.100000    17    -61.196684   0.002250 -61.194434 -60.874273   0.001919   
0.140271    18    -61.196998   0.003255 -61.193743 -60.874584   0.002871   
0.196760    19    -61.197384   0.004731 -61.192653 -60.874965   0.004306   
0.275997    20    -61.197828   0.006836 -61.190993 -60.875397   0.006370   
0.387100    21    -61.198272   0.009775 -61.188496 -60.875818   0.009220   
0.543100    22    -61.198533   0.013843 -61.184689 -60.876039   0.013105   
0.761700    23    -61.198134   0.019335 -61.178799 -60.875571   0.018405   
1.068500    24    -61.196878   0.025766 -61.171112 -60.874202   0.024887   
1.498800    25    -61.194984   0.032509 -61.162475 -60.872153   0.032074   
2.102400    26    -61.192439   0.039725 -61.152713 -60.869405   0.039960   
2.949000    27    -61.189388   0.047639 -61.141747 -60.866103   0.048359   
4.136600    28    -61.185806   0.056973 -61.128832 -60.862200   0.057776   
5.802500    29    -61.181559   0.068630 -61.112929 -60.857552   0.069098   
8.139200    30    -61.176500   0.083527 -61.092973 -60.851972   0.083369   
11.417000   31    -61.170499   0.102636 -61.067863 -60.845333   0.101068   
16.014700   32    -61.163495   0.127090 -61.036403 -60.837562   0.122860   
22.464000   33    -61.155564   0.158280 -60.997284 -60.828802   0.150716   
31.510500   34    -61.146970   0.197881 -60.949090 -60.819448   0.187644   
44.200100   35    -61.137901   0.247538 -60.890363 -60.809950   0.234648   
62.000000   36    -61.134429   0.304142 -60.830286 -60.807142   0.287051   
85.775000   37    -61.152400   0.360910 -60.791490 -60.828205   0.338804   
109.550000  38    -61.192701   0.412804 -60.779896 -60.872843   0.387514   
133.325000  39    -61.244101   0.485817 -60.758283 -60.928222   0.458500   
157.100000  40    -61.296048   0.596845 -60.699202 -60.983493   0.568490   
180.875000  41    -61.349486   0.753792 -60.595695 -61.039739   0.726776   
204.650000  42    -61.415654   1.017832 -60.397822 -61.109314   0.998656   
228.425000  43    -61.530076   1.601450 -59.928626 -61.230932   1.607725   
252.200000  44    -61.706041   2.601997 -59.104045 -61.418028   2.608892   
275.975000  45    -61.954737   4.070990 -57.883746 -61.680009   4.010538   
299.750000  46    -62.284616   6.021788 -56.262827 -62.023372   5.879774   
323.525000  47    -62.701483   8.429407 -54.272076 -62.452129   8.174785   
347.300000  48    -63.209652  11.240297 -51.969355 -62.972620  10.772185   
371.075000  49    -63.808142  14.368465 -49.439678 -63.583520  13.710793   
394.850000  50    -64.468754  17.632616 -46.836138 -64.255378  16.874843 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')